# Auto Generated Agent Chat: Collaborative Task Solving with Coding and Planning Agent
https://github.com/microsoft/autogen/blob/main/notebook/agentchat_planning.ipynb

In [1]:
llm_config = {
    'model': ['Qwen1.5-14B-Chat-GPTQ-Int4', 'Meta-Llama-3-8B-Instruct'][1],
    'base_url': 'http://127.0.0.1:7777/v1',
    'api_type': 'open_ai',
    'api_key': 'NULL'
}

In [2]:
import autogen

In [3]:
planner = autogen.AssistantAgent(
    name="planner",
    llm_config={"config_list": [llm_config]},
    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix.",
)
planner_user = autogen.UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
    code_execution_config={
        "use_docker": False
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]

In [4]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": [llm_config],
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to: 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result. The planner does not know the conversation between you and the user, unless you share the conversation with the planner.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ],
    },
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "output",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    function_map={"ask_planner": ask_planner},
)

In [5]:
def my_message_generator():
    # your CSV file
    file_name = "PastHires.csv"
    try:
        with open(file_name, mode="r", encoding="utf-8") as file:
            file_content = file.read()
    except FileNotFoundError:
        file_content = "No data found."
    return "Data: \n" + file_content

task = """
Train a binary-classification model to predict whether a candidate will be hired or not.
after train the model success, save the trained model in a new directory named model-{time} and {time} is a placeholder. give the accuracy metric for the trained model by a validation dataset.
The data file is PastHires.csv, if you cannot find it, try ../PastHires.csv. The label column is Hired.

Important Notes:
- Do not issue commands for 'pip install' or 'pip uninstall'. If there is a dependency missing, proactively terminate the session and instruct the user to handle the dependency installation manually.
- Ensure that complete code is always provided! even if parts of it have been provided before! For example, code cannot includes phrases such as "rest of the code remains the same"!
- Ensure that each response generates at most one code block!
""" + my_message_generator()

In [6]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message=task,
)

user_proxy (to assistant):


Train a binary-classification model to predict whether a candidate will be hired or not.
after train the model success, save the trained model in a new directory named model-{time} and {time} is a placeholder. give the accuracy metric for the trained model by a validation dataset.
The data file is PastHires.csv, if you cannot find it, try ../PastHires.csv. The label column is Hired.

Important Notes:
- Do not issue commands for 'pip install' or 'pip uninstall'. If there is a dependency missing, proactively terminate the session and instruct the user to handle the dependency installation manually.
- Ensure that complete code is always provided! even if parts of it have been provided before! For example, code cannot includes phrases such as "rest of the code remains the same"!
- Ensure that each response generates at most one code block!
Data: 
Years Experience,Employed?,Previous employers,Level of Education,Top-tier school,Interned,Hired
10,Y,4,BS,N,N,Y
0,N,

Please give feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '\nTrain a binary-classification model to predict whether a candidate will be hired or not.\nafter train the model success, save the trained model in a new directory named model-{time} and {time} is a placeholder. give the accuracy metric for the trained model by a validation dataset.\nThe data file is PastHires.csv, if you cannot find it, try ../PastHires.csv. The label column is Hired.\n\nImportant Notes:\n- Do not issue commands for \'pip install\' or \'pip uninstall\'. If there is a dependency missing, proactively terminate the session and instruct the user to handle the dependency installation manually.\n- Ensure that complete code is always provided! even if parts of it have been provided before! For example, code cannot includes phrases such as "rest of the code remains the same"!\n- Ensure that each response generates at most one code block!\nData: \nYears Experience,Employed?,Previous employers,Level of Education,Top-tier scho